In [426]:
from pymongo import MongoClient
from bson.objectid import ObjectId
import requests
import json

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity as distance


client = MongoClient()
db = client["API"]
m = db['message']
USER = db['user']
chat = db['chat']


In [440]:
users_list = ['5de391d57f6e91bd6cf56455',
 '5de391d57f6e91bd6cf56456',
 '5de391d57f6e91bd6cf56457',
 '5de391d57f6e91bd6cf56458']
if '5de391d57f6e91bd6cf56458' in users_list:
    print("good")
else:
    raise TypeError("The user doesnt belong to this chat")

good


# RECOMENDATIONS

In [394]:
info = {}
x=list(m.find({},{'message':1,'user_id':1,'_id':0}))
for i in x:
    if i['user_id'] not in info.keys():
        info[i['user_id']] = i['message']
    else:
        info[i['user_id']] = info[i['user_id']]+" "+i['message']
        

In [395]:
count_vectorizer = CountVectorizer(stop_words='english')
count_vectorizer = CountVectorizer()
sparse_matrix = count_vectorizer.fit_transform(info.values())


In [407]:
info_matrix = sparse_matrix.todense()
df = pd.DataFrame(info_matrix, 
                  columns=count_vectorizer.get_feature_names(), 
                  index=info.keys())


In [409]:
similarity_matrix = distance(df, df)
sim_df = pd.DataFrame(similarity_matrix, columns=info.keys(), index=info.keys())
display(sim_df)

,5de391d57f6e91bd6cf56455,5de391d57f6e91bd6cf56456,5de391d57f6e91bd6cf56457,5de3aad67f6e91bd6cf56461,5de3aad67f6e91bd6cf5645f
5de391d57f6e91bd6cf56455,1.00,0.000000,0.000000,0.75,0.0
5de391d57f6e91bd6cf56456,0.00,1.000000,0.218218,0.00,0.0
5de391d57f6e91bd6cf56457,0.00,0.218218,1.000000,0.00,0.0
5de3aad67f6e91bd6cf56461,0.75,0.000000,0.000000,1.00,0.0
5de3aad67f6e91bd6cf5645f,0.00,0.000000,0.000000,0.00,1.0


In [398]:
chosen_user = '5de391d57f6e91bd6cf56456'
recom = sim_df[chosen_user].sort_values(ascending=False)[1:]

In [425]:
g = {"user0": "5de391d57f6e91bd6cf56456", "user1": "5de3aad67f6e91bd6cf5645f", "user2": "5de3aad67f6e91bd6cf56461"}

for i in g.values():
    print(i)

5de391d57f6e91bd6cf56456
5de3aad67f6e91bd6cf5645f
5de3aad67f6e91bd6cf56461


In [406]:
recommendations = []
for u in users[:3]:
    recommendations.append(list(USER.find({'_id':ObjectId(u)}))[0])
    
recommendations

[{'_id': ObjectId('5de391d57f6e91bd6cf56457'), 'username': 'Cristina'},
 {'_id': ObjectId('5de3aad67f6e91bd6cf5645f'), 'username': 'Gonzalo'},
 {'_id': ObjectId('5de3aad67f6e91bd6cf56461'), 'username': 'Maria'}]